In [61]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

In [62]:
#Importing and formatting the df
stocks = pd.read_csv('adjprice.csv')
stocks['Date'] = pd.to_datetime(stocks['Date'], format='%Y%m%d')
stocks

,Date,0111145D US Equity,0202445Q US Equity,0203524D US Equity,0226226D US Equity,0376152D US Equity,0440296D US Equity,0544749D US Equity,0574018D US Equity,0598884D US Equity,...,YNR US Equity,YRCW US Equity,YUM US Equity,YUMC US Equity,ZBH US Equity,ZETHQ US Equity,ZION US Equity,ZRN US Equity,ZTS US Equity,ZTS-W US Equity
0,1993-09-07,13.2719,13.6829,8.4429,8.1042,11.000,57.3245,17.8887,6.8315,28.1246,...,NaN,144439.5121,NaN,NaN,NaN,6.875,7.1675,30.7970,NaN,NaN
1,1993-09-08,13.3263,13.5315,8.2147,7.9590,11.000,57.2096,17.8064,6.8315,27.5051,...,NaN,143691.1208,NaN,NaN,NaN,6.750,6.9766,30.0188,NaN,NaN
2,1993-09-09,13.7070,13.3800,8.7852,8.0627,11.125,59.1625,17.6831,6.8315,27.7529,...,NaN,143691.1208,NaN,NaN,NaN,6.625,6.9554,30.0188,NaN,NaN
3,1993-09-10,13.3807,13.4810,9.4127,8.0368,11.125,59.6220,17.6420,6.8773,27.5051,...,NaN,145187.9033,NaN,NaN,NaN,6.750,7.2099,29.6852,NaN,NaN
4,1993-09-11,13.3807,13.4810,9.4127,8.0368,11.125,59.6220,17.6420,6.8773,27.5051,...,NaN,145187.9033,NaN,NaN,NaN,6.750,7.2099,29.6852,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9454,2019-07-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.3500,114.02,45.31,134.50,NaN,45.1600,NaN,115.11,NaN
9455,2019-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.3500,114.02,45.31,134.50,NaN,45.1600,NaN,115.11,NaN
9456,2019-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.1800,114.10,45.43,134.53,NaN,44.6500,NaN,115.55,NaN
9457,2019-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.1800,113.24,44.00,136.67,NaN,45.4800,NaN,115.79,NaN


In [63]:
# creating date storage
dates = []
# appending dates as tuples
for i in range(1994, 2019):
    if i+6 <= 2018:
        StartDate = f"{i}/01/01"
        EndDate = f"{i+5}/12/31"
        TestDate = f"{i+6}/01/01"
        TestEndDate = f"{i+6}/12/31"
        dates.append((StartDate, EndDate, TestDate, TestEndDate))
        print("start date:", StartDate, "end date:", EndDate, "test date:", TestDate, "test end date:", TestEndDate)

start date: 1994/01/01 end date: 1999/12/31 test date: 2000/01/01 test end date: 2000/12/31
start date: 1995/01/01 end date: 2000/12/31 test date: 2001/01/01 test end date: 2001/12/31
start date: 1996/01/01 end date: 2001/12/31 test date: 2002/01/01 test end date: 2002/12/31
start date: 1997/01/01 end date: 2002/12/31 test date: 2003/01/01 test end date: 2003/12/31
start date: 1998/01/01 end date: 2003/12/31 test date: 2004/01/01 test end date: 2004/12/31
start date: 1999/01/01 end date: 2004/12/31 test date: 2005/01/01 test end date: 2005/12/31
start date: 2000/01/01 end date: 2005/12/31 test date: 2006/01/01 test end date: 2006/12/31
start date: 2001/01/01 end date: 2006/12/31 test date: 2007/01/01 test end date: 2007/12/31
start date: 2002/01/01 end date: 2007/12/31 test date: 2008/01/01 test end date: 2008/12/31
start date: 2003/01/01 end date: 2008/12/31 test date: 2009/01/01 test end date: 2009/12/31
start date: 2004/01/01 end date: 2009/12/31 test date: 2010/01/01 test end date:

In [64]:
for date_tuple in dates:

    # first, subsetting stocks dataframe to just include training and testing period, i.e. 7 year period
    stocks_total = stocks[(stocks["Date"] >= date_tuple[0]) & (stocks["Date"] <= date_tuple[3])]
    stocks_total.dropna(axis=1, inplace=True)

    # training data
    stocks_train_subset = stocks_total[(stocks_total["Date"] >= date_tuple[0]) & (stocks_total["Date"] <= date_tuple[1])]
    stocks_train_subset.dropna(axis=1,inplace=True)

    #testing data
    stocks_test_subset = stocks_total[(stocks_total["Date"] >= date_tuple[2]) & (stocks_total["Date"] <= date_tuple[3])]
    stocks_test_subset.dropna(axis=1, inplace=True)
    
    # calculating return
    stocks_return = stocks_train_subset.iloc[-1,1:]/stocks_train_subset.iloc[0,1:]-1
    log_return = np.log(1 + stocks_return.pct_change())
    
    sort_stocks = pd.Series.sort_values(log_return, ascending = False)
    
    top_return = sort_stocks.head(10)
    
    # obtaining the stock names
    stock_list = []
    for i in range(len(top_return)):
        stock_list.append(top_return.index[i])
        
    # create a new dataframe with the first column being the dates
    new_portfolio = pd.DataFrame(stocks_test_subset['Date'],columns=['Date'])
    # go through all the columns in our test data frame
    for col in stocks_test_subset.columns:
        # check if this column is in the names for our top stocks
        if col in stock_list: 
            # if so, add the stock prices to the new dataframe
            new_portfolio = pd.concat([new_portfolio,stocks_test_subset[col]],axis=1) 
    
    # defining initial holdings list
    initial_holdings = []
    print(new_portfolio)

           Date  3153670Q US Equity  BIIB US Equity  CLX US Equity  \
2307 2000-01-01             95.4167          30.166         30.451   
2308 2000-01-02             95.4167          30.166         30.451   
2309 2000-01-03             94.6667          28.631         29.733   
2310 2000-01-04             86.8750          26.693         28.939   
2311 2000-01-05             84.7917          28.554         29.695   
...         ...                 ...             ...            ...   
2668 2000-12-27             93.0000          61.388         19.918   
2669 2000-12-28             95.4375          60.409         20.612   
2670 2000-12-29             87.5000          58.202         21.883   
2671 2000-12-30             87.5000          58.202         21.883   
2672 2000-12-31             87.5000          58.202         21.883   

      DELL US Equity  FMC US Equity  GTW US Equity  JNJ US Equity  \
2307         48.8721         6.8273        72.0625         29.005   
2308         48.8721 

           Date  CCI US Equity  CHK US Equity  CMX US Equity  DE US Equity  \
4134 2005-01-01        13.9886        13.7526        34.5853       27.6011   
4135 2005-01-02        13.9886        13.7526        34.5853       27.6011   
4136 2005-01-03        13.7952        12.8941        33.9011       27.0595   
4137 2005-01-04        13.5767        12.7857        34.6029       26.4140   
4138 2005-01-05        13.9129        12.6857        35.7431       26.1580   
...         ...            ...            ...            ...           ...   
4494 2005-12-27        22.5297        25.7180        45.3828       25.9639   
4495 2005-12-28        22.5381        26.3225        45.6109       26.0964   
4496 2005-12-29        22.5213        26.3729        45.5495       25.9904   
4497 2005-12-30        22.6222        26.6416        45.4267       25.7784   
4498 2005-12-31        22.6222        26.6416        45.4267       25.7784   

      DVA US Equity  EXC US Equity  MTB US Equity  NKE US Equit

           Date  1288453D US Equity  AKAM US Equity  ALXN US Equity  \
5595 2009-01-01              16.765           15.09          18.095   
5596 2009-01-02              17.405           15.74          18.065   
5597 2009-01-03              17.405           15.74          18.065   
5598 2009-01-04              17.405           15.74          18.065   
5599 2009-01-05              17.275           16.18          18.500   
...         ...                 ...             ...             ...   
5955 2009-12-27              19.265           25.74          24.235   
5956 2009-12-28              19.530           25.89          24.220   
5957 2009-12-29              19.565           25.71          24.140   
5958 2009-12-30              19.460           25.72          24.325   
5959 2009-12-31              19.200           25.34          24.410   

      CELG US Equity  CTSH US Equity  GD US Equity  MNST US Equity  \
5595          27.640          8.8772        45.506           5.588   
5596   

           Date  BAC US Equity  COO US Equity  EXPE US Equity  FTI US Equity  \
7056 2013-01-01        10.8660        92.2698          58.223        41.8521   
7057 2013-01-02        11.2591        94.9437          59.938        42.5068   
7058 2013-01-03        11.1936        94.7142          59.540        42.1258   
7059 2013-01-04        11.3340        95.2230          61.085        42.6437   
7060 2013-01-05        11.3340        95.2230          61.085        42.6437   
...         ...            ...            ...             ...            ...   
7416 2013-12-27        14.7098       125.3624          65.797        50.7151   
7417 2013-12-28        14.7098       125.3624          65.797        50.7151   
7418 2013-12-29        14.7098       125.3624          65.797        50.7151   
7419 2013-12-30        14.5878       124.7734          65.874        50.3633   
7420 2013-12-31        14.6159       123.6254          66.659        51.0180   

      HRL US Equity  LB US Equity  MMC 

           Date  AET US Equity  AVGO US Equity  BIG US Equity  ES US Equity  \
8882 2018-01-01        178.457        251.4695        54.9914       61.5686   
8883 2018-01-02        178.754        261.3658        54.7466       61.5393   
8884 2018-01-03        179.852        264.2241        53.8064       60.9936   
8885 2018-01-04        181.405        264.3122        55.4811       60.1361   
8886 2018-01-05        183.819        265.8783        56.7445       60.2432   
...         ...            ...             ...            ...           ...   
9242 2018-12-27        212.700        246.2679        27.8807       63.8167   
9243 2018-12-28        212.700        248.9579        27.5178       63.7083   
9244 2018-12-29        212.700        248.9579        27.5178       63.7083   
9245 2018-12-30        212.700        248.9579        27.5178       63.7083   
9246 2018-12-31        212.700        249.6353        28.3713       64.0728   

      FLT US Equity  GNW US Equity  NVDA US Equity 

In [65]:
rF = 0.01 # Assume risk free rate is 0.1%
number_of_portfolios = 3000

# Initialize new variables to populate with the results of the simulation.
portfolio_returns = []
portfolio_risk = []
sharpe_ratio_portfolio = []
portfolio_weights = []

# Create a loop with number of iterations equal to the number of portfolios.
for i in range(number_of_portfolios):
    random_weights = np.random.random_sample(number_of_portfolios)
    rebalanced_weights = random_weights/np.sum(random_weights) # make weights sum to 1
    print(rebalanced_weights)
    
    #annualized return based on the asset allocation
    annualized_return = np.sum(pd.Series(stocks_total.mean() * rebalanced_weights * 252))
    portfolio_returns.append(annualized_return) #save returns for each specific portfolio into our list

    #calculate variance/risk
    matrix_covariance = returns.cov()
    matrix_covariance['Bonds'] = matrix_covariance['Bonds'] * 250
    matrix_covariance['BTC'] = matrix_covariance['BTC'] * 365
    matrix_covariance['Stocks'] = matrix_covariance['Stocks'] * 250
    portfolio_variance = np.dot(weights.T, np.dot(matrix_covariance, rebalanced_weights))
    portfolio_std = np.sqrt(portfolio_variance)
    portfolio_risk.append(portfolio_std)

    #calculate Sharpe Ratio
    sharpe_ratio = ((annualized_return - rF) /portfolio_std)
    sharpe_ratio_portfolio.append(sharpe_ratio)

    #store weights, because we're interested in the weights of the portfolio with the
    #highest Sharpe ratio
    portfolio_weights.append(weights)

    portfolio_returns = np.array(portfolio_returns)
    portfolio_risk = np.array(portfolio_risk)
    sharpe_ratio_portfolio = np.array(sharpe_ratio_portfolio)

    # Plot all of the portfolio risk vs return to demonstrate the Efficient Frontier.
    # Note also that the color bar varies with Sharpe Ratio, to show the "best" portfolios.
    plt.figure(figsize = (12,6))
    plt.scatter(portfolio_risk, portfolio_returns, c = portfolio_returns / portfolio_risk, cmap = 'inferno')
    plt.xlabel('Volatility')
    plt.ylabel('Returns')
    plt.colorbar(label = 'Sharpe Ratio')

    # Save the portfolio metrics into a dataframe for further exploration.
    portfolio_metrics = [portfolio_returns, portfolio_risk, sharpe_ratio_portfolio, portfolio_weights]
    portfolio_dfs = pd.DataFrame(portfolio_metrics, )
    portfolio_dfs = portfolio_dfs.T
    portfolio_dfs.columns = ['Returns', 'Risk', 'Sharpe Ratio', 'Weights']

    # Find the portfolio with the highest Sharpe Ratio
    highest_sharpe_ratio = portfolio_dfs.iloc[portfolio_dfs['Sharpe Ratio'].idxmax()]

    # Find the portfolio with the highest returns
    max_returns = portfolio_dfs.iloc[portfolio_dfs['Returns'].idxmax()]

    # Find the portfolio with the lowest risk
    min_risk = portfolio_dfs.iloc[portfolio_dfs['Risk'].idxmin()]

[0.00062155 0.00028691 0.00019322 ... 0.00029957 0.00051578 0.00029013]
           Date  A US Equity  AA US Equity  AAP US Equity  AAPL US Equity  \
6690 2012-01-01      23.3620       18.3204         68.643         51.0979   
6691 2012-01-02      23.3620       18.3204         68.643         51.0979   
6692 2012-01-03      24.3986       19.5488         68.120         51.8839   
6693 2012-01-04      24.2047       20.0148         69.717         52.1628   
6694 2012-01-05      24.7464       19.8241         70.161         52.7419   
...         ...          ...           ...            ...             ...   
9242 2018-12-27      66.0353       27.1600        154.573        154.8867   
9243 2018-12-28      65.6808       26.6000        155.343        154.9660   
9244 2018-12-29      65.6808       26.6000        155.343        154.9660   
9245 2018-12-30      65.6808       26.6000        155.343        154.9660   
9246 2018-12-31      67.1745       26.5800        157.341        156.4638   

   

ValueError: operands could not be broadcast together with shapes (592,) (3000,) 